<img src='letthembreathespace.jpg' width="480" height="370" align="left">

In [1]:
#! pip install numpy matplotlib scipy sklearn hmmlearn simplejson eyed3 pydub --user
#!pip install pydub --user
#!pip install ffmpeg --user
#!pip install deepspeech==0.2.0a8 --user
#!pip install sox  --user

#import speech_recognition as sr

In [2]:
import numpy as np
import os
import subprocess
import uuid
import sys

In [3]:
import warnings
warnings.filterwarnings("ignore")
import scipy.io.wavfile as wav

In [4]:
from deepspeech.model import Model

In [5]:
import ffmpeg
from pydub import AudioSegment
from pydub.playback import play

Create Directories

In [6]:
AudiodirName = './SoundFiles'
ModeldirName = './models'

DirList=[AudiodirName,ModeldirName]

In [7]:
for i in DirList:
    try:
        # Create target Directory
        os.mkdir(i)
        print("Directory " ,i,  " Created ") 
    except:
        print("Directory " ,i,  " already exists")

#os.chdir(AudiodirName)
#print('Moved to directory',os.getcwd())

Directory  ./SoundFiles  already exists
Directory  ./models  already exists


Download Audio Book and Extract it to Audio Folder

In [ ]:
#!wget http://www.archive.org/download//letembreathespaceversion2_1809_librivox/letembreathespaceversion2_1809_librivox_64kb_mp3.zip

In [8]:
import zipfile
Save_Location=AudiodirName
Zip_Location='./letembreathespaceversion2_1809_librivox_64kb_mp3.zip'
#Extracting Zip File
zip_ref = zipfile.ZipFile(Zip_Location, 'r')


#Preview files within zip
for name in zip_ref.namelist():
    print('%s' % (name))
    print('Saved in location:\t',Save_Location)
    
zip_ref.extractall(Save_Location)
zip_ref.close()

letembreathspace_03_delrey_64kb.mp3
Saved in location:	 ./SoundFiles
letembreathspace_01_delrey_64kb.mp3
Saved in location:	 ./SoundFiles
letembreathspace_04_delrey_64kb.mp3
Saved in location:	 ./SoundFiles
letembreathspace_02_delrey_64kb.mp3
Saved in location:	 ./SoundFiles
letembreathspace_05_delrey_64kb.mp3
Saved in location:	 ./SoundFiles


Split Audio File based on Decibal Level and Silence Time (miliseconds)

*We are trying to achieve less than a minute in Audio Length for each file. This is because DeepSpeech has a difficult time parsing text to audio files larger than what it was trained on. For simplicity each seperation is restricted to less than a minute.*

In [9]:
from pydub import AudioSegment
from pydub.silence import split_on_silence

sound = AudioSegment.from_file(AudiodirName+"/letembreathspace_01_delrey_64kb.mp3", format="mp3")
sound = sound.set_frame_rate(16000).set_channels(1)
chunks = split_on_silence(
    sound,
    # split on silences longer than 1000ms (1 sec)
    min_silence_len=1000,

    # anything under -16 dBFS is considered silence
    silence_thresh=-64, 

    # keep 200 ms of leading/trailing silence
    keep_silence=200
)

# now recombine the chunks so that the parts are at least 90 sec long
target_length = 90 * 1000
output_chunks = [chunks[0]]
for chunk in chunks[1:]:
    if len(output_chunks[-1]) < target_length:
        output_chunks[-1] += chunk
    else:
        # if the last output chunk is longer than the target length,
        # we can start a new one
        output_chunks.append(chunk)

In [10]:
FileList=[]
for i, chunk in enumerate(chunks):
    #print(i)
    #print(chunk)
    chunk.export(AudiodirName+"/chunk{0}_16bit.wav".format(i), format="wav", bitrate="16k")
    FileList.append(AudiodirName+"/chunk{0}_16bit.wav".format(i))

In [11]:
chunks[:3]

In [12]:
Test_List=FileList[:1]

Sampling only 15 of the audio files that were split for testing purposes

In [13]:
Test_List

['./SoundFiles/chunk0_16bit.wav']

Create a list of audio files to be translated into text

In [14]:
ListString=''
for i in Test_List:
    ListString=ListString+i+' '
    with open('audio_list.txt',"a+") as f:
        f.write(str(i)+'\n')

In [15]:
Test_List

['./SoundFiles/chunk0_16bit.wav']

In [17]:
TextOut={}
for _file_id in range(0,len(Test_List)): 
    # Get Audio Filename
    vf = Test_List[_file_id]    
    print('file: ',vf)
    print(' ')
    file_path,file_name = os.path.split(vf)
#    folder_name = AudiodirName
#    print(folder_name)
#    try:
#        os.makedirs(folder_name)
#    except:
#        print("Directory %s exists \n"%folder_name)
                #model location                   alphabet file
    ds = Model(ModeldirName+'/output_graph.pb', 26, 9, ModeldirName+'/alphabet.txt', 500)
    fs, audio = wav.read(vf)
    processed_data=ds.stt(audio,fs)
    processed_data=ds.stt(audio.flatten(),fs)
    TextOut[file_name]=processed_data
    
    try:
        print('\nDeepSpeech says, "...'+str(processed_data)+'..."\n')
    except:
        print("print statement didn't work")



file:  ./SoundFiles/chunk0_16bit.wav
 

DeepSpeech says, "...chaper one of lot om breethe space this is a lebrivox recording all leberhoxks recordings are in the public domain or more information nor e volunteer please visit lebri vox don orp recording by fill shon aber in the byus of louisziana..."



In [18]:
TextOut

{'chunk0_16bit.wav': 'chaper one of lot om breethe space this is a lebrivox recording all leberhoxks recordings are in the public domain or more information nor e volunteer please visit lebri vox don orp recording by fill shon aber in the byus of louisziana'}

In [11]:
#import subprocess
#from subprocess import call
#subprocess.call(['ffmpeg','-i','12_sec_mono_16bit.wav -f segment -segment_time 15 -c copy out%03d.wav'])

In [12]:
#ffmpeg -i 12_sec_mono_16bit.wav -f segment -segment_time 15 -c copy out%03d.wav

In [13]:
#sound = AudioSegment.from_file("12_sec_mono_16bit.wav", format="wav")
#play(sound)

In [28]:
#t1 = t1 * 60000 #Works in milliseconds, 60000 in 1 minute
#t2 = t2 * 60000

In [ ]:
#newAudio = AudioSegment.from_wav("17401-chap_10_mono_16bit.wav")
#newAudio = newAudio[t1:t2]
#newAudio.export('newSong.wav', format="wav") #Exports to a wav file in the current path.

In [ ]:
#file_list = []
# open file and read the content in a list
#with open('audio_list.txt', 'r') as filehandle:  
#    file_list = [current_file.rstrip() for current_file in filehandle.readlines()]